<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Все-задания-нужно-выполнить-только-с-помощью-sql" data-toc-modified-id="Все-задания-нужно-выполнить-только-с-помощью-sql-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Все задания нужно выполнить только с помощью sql</a></span></li></ul></div>

In [77]:
import sqlite3
import pandas as pd

sqlite3.sqlite_version

'3.30.0'

In [78]:
con = sqlite3.connect(':memory:')

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [79]:
data = pd.read_csv('./csv_for_sql/boston.csv')
data.head()

CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2

In [80]:
data.to_sql('boston', con, index=False, if_exists='replace')

In [146]:
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

CRIM    ZN  INDUS  CHAS    NOX     RM   AGE     DIS  RAD    TAX  \
0  0.00632  18.0   2.31   0.0  0.538  6.575  65.2  4.0900  1.0  296.0   
1  0.02731   0.0   7.07   0.0  0.469  6.421  78.9  4.9671  2.0  242.0   
2  0.02729   0.0   7.07   0.0  0.469  7.185  61.1  4.9671  2.0  242.0   
3  0.03237   0.0   2.18   0.0  0.458  6.998  45.8  6.0622  3.0  222.0   
4  0.06905   0.0   2.18   0.0  0.458  7.147  54.2  6.0622  3.0  222.0   

   PTRATIO       B  LSTAT  MEDV  
0     15.3  396.90   4.98  24.0  
1     17.8  396.90   9.14  21.6  
2     17.8  392.83   4.03  34.7  
3     18.7  394.63   2.94  33.4  
4     18.7  396.90   5.33  36.2

In [82]:
tasks = 0

### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [83]:
tasks+=1
pd.read_sql(
    '''
    with CHECK_EMPTY as (
        SELECT 
            case when CRIM is null then 1 else 0 end as CRIM
            , case when ZN is null then 1 else 0 end as ZN
            , case when  INDUS is null then 1 else 0 end as INDUS
            , case when CHAS is null then 1 else 0 end as CHAS
            , case when NOX is null then 1 else 0 end as NOX
        FROM boston)
    select 
        sum(CRIM) CRIM
        , sum(ZN) ZN
        , sum(INDUS) INDUS
        , sum(CHAS) CHAS
        , sum(NOX) NOX
    from CHECK_EMPTY
    ''',
    con,
)

CRIM  ZN  INDUS  CHAS  NOX
0     0   0      0     0    0

2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [84]:
tasks+=1
pd.read_sql(
    '''
        SELECT 
           count(distinct(CRIM)) as CRIM
           , count(distinct(ZN)) as ZN
           , count(distinct(INDUS)) as INDUS
           , count(distinct(CHAS)) as CHAS
           , count(distinct(NOX)) as NOX
        FROM boston
    ''',
    con,
)

CRIM  ZN  INDUS  CHAS  NOX
0   504  26     76     2   81

3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [145]:
tasks+=1
pd.read_sql(
    '''
    with medians as (
        SELECT 'CRIM' as COLUMN_NAME, AVG(CRIM) as median_value
        FROM (
                SELECT CRIM
                FROM boston
                ORDER BY CRIM
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'ZN' as COLUMN_NAME, AVG(ZN) as median_value
        FROM (
                SELECT ZN
                FROM boston
                ORDER BY ZN
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'INDUS' as COLUMN_NAME, AVG(INDUS) as median_value
        FROM (
                SELECT INDUS
                FROM boston
                ORDER BY INDUS
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'CHAS' as COLUMN_NAME, AVG(CHAS) as median_value
        FROM (
                SELECT CHAS
                FROM boston
                ORDER BY CHAS
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston))
        UNION ALL
        SELECT 'NOX' as COLUMN_NAME, AVG(NOX) as median_value
        FROM (
                SELECT NOX
                FROM boston
                ORDER BY NOX
                LIMIT 2 - (SELECT COUNT(*) FROM boston) % 2
                OFFSET (SELECT (COUNT(*) - 1) / 2 FROM boston)))
    , mins as(
        SELECT distinct
               'CRIM' as COLUMN_NAME
                , min(CRIM) OVER() as min_value
        FROM boston b
        UNION ALL
        SELECT distinct
               'ZN' as COLUMN_NAME
                , min(ZN) OVER() as min_value
                FROM boston b
        UNION ALL
        SELECT distinct
               'INDUS' as COLUMN_NAME                
                , min(INDUS) OVER() as min_value
                FROM boston b
        UNION ALL
        SELECT distinct
               'CHAS' as COLUMN_NAME
                , min(CHAS) OVER() as min_value
                FROM boston b
        UNION ALL
        SELECT distinct
               'NOX' as COLUMN_NAME
                , min(NOX) OVER() as min_value
        FROM boston b 
    )
    
    SELECT distinct
           mins.COLUMN_NAME
            , mins.min_value
            , medians.median_value
    FROM mins  
        join medians on medians.COLUMN_NAME=mins.COLUMN_NAME
    where min_value=median_value
        
    ''',
    con,
)

COLUMN_NAME  min_value  median_value
0          ZN        0.0           0.0
1        CHAS        0.0           0.0

Выводы: 

Не менее чем в половине рассматриваемых кейсов доля земель под жилую застройку не зонирована под участки площадью более 25 000 кв. футов.

Не менее чем в половине рассматриваемых кейсов участки не граничат с рекой.

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [167]:
pd.read_sql(
    '''
    select avg(RM) FROM boston where MEDV =(SELECT MAX(MEDV) over() FROM boston)
    ''',
    con,
)

avg(RM)
0    7.484

In [166]:
tasks+=1
pd.read_sql(
    '''
   -- select avg(RM)
   -- FROM boston 
   -- where MEDV =(SELECT MAX(MEDV) over() FROM boston)
    select 
        MEDV
        , RM
        , avg(RM) over(ORDER BY MEDV RANGE CURRENT ROW) as AVG_RM_MAX
    FROM boston 

    ''',
    con,
)

MEDV     RM  AVG_RM_MAX
0     5.0  5.453      5.5680
1     5.0  5.683      5.5680
2     5.6  5.987      5.9870
3     6.3  5.852      5.8520
4     7.0  4.519      4.9665
..    ...    ...         ...
501  50.0  4.970      7.4840
502  50.0  6.683      7.4840
503  50.0  7.016      7.4840
504  50.0  6.216      7.4840
505  50.0  5.875      7.4840

[506 rows x 3 columns]

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех значений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [ ]:
tasks+=1
pd.read_sql(
    '''
    SELECT * FROM boston
    limit 5
    ''',
    con,
)

In [ ]:
if tasks==8:
    print('Выполнены все задания 🙂')
else:
    print('Выполнены не все задания! 🙀')